In [673]:
import numpy as np

def confidence_noise_determination(img, c, v, beta, r):
    """
    Parameters:
    M: deterministic mechanism
    D: data distribution
    m: sampling complexity
    c, v, beta: constants

    Returns:
    Gaussian covariance matrix ΣB
    """

    # Step 6: Calculate empirical mean and empirical covariance estimation
    mu_hat = np.mean(img, axis=0)
    empirical_cov = np.var(img, axis=0)

    # Step 7: Apply singular value decomposition (SVD)
    U, Lambda, UT = np.linalg.svd(empirical_cov)
    # print(Lambda)
    
    # Step 8: Determine the maximal index j0
    if c > np.max(Lambda):
        condition = False
    else:
        j0 = np.argmin([lambd for lambd in Lambda if lambd > c])
    
        # Step 9: Check the condition
        condition = np.min([np.abs(Lambda[j] - Lambda[l])for j in range(j0 + 1) for l in range(j + 1, len(Lambda))]) > r * np.sqrt(len(Lambda) * c) + 2 * c
    
    if condition:
        # Step 10 & 11: Determine the j-th element of a diagonal matrix ΛB
        sqrt_sum = sum(np.sqrt(Lambda[j] + 10 * c * v/ beta) for j in range(len(Lambda)))
        Lambda_B = [2 * v / (np.sqrt(Lambda[j] + 10 * c * v/ beta) * sqrt_sum) for j in range(len(Lambda))]
        ΛB = np.diag(Lambda_B)

        # Step 13: Determine the Gaussian noise covariance
        ΣB = U @ np.linalg.inv(ΛB) @ UT

    else:
        # Step 15: Determine the Gaussian noise covariance
        ΣB = (sum(Lambda) + len(Lambda) * c) / (2 * v) * np.eye(len(Lambda))

    return ΣB


In [674]:
import os
from PIL import Image, ImageOps 

In [675]:
img_dir = '/data/local/ml01/qipan/exp_celeba/CG_inf_smile_samples_1'
img = [ImageOps.grayscale(Image.open(os.path.join(img_dir, f))) for f in os.listdir(img_dir) if f.endswith('.png')]
img = np.array([np.array(i) for i in img])

In [676]:
img.shape

(1000, 64, 64)

In [677]:
img = (img - np.min(img, axis=0))/ (np.max(img, axis=0)- np.min(img, axis=0))

In [678]:
empirical_cov = np.var(img, axis=0)

In [679]:
empirical_cov.shape

(64, 64)

In [680]:
empirical_cov

array([[0.11790869, 0.11739077, 0.11939488, ..., 0.11712946, 0.11684583,
        0.11841538],
       [0.11736821, 0.11817271, 0.1195778 , ..., 0.11588179, 0.11610222,
        0.11672638],
       [0.11950845, 0.1186466 , 0.12033208, ..., 0.1148446 , 0.1136201 ,
        0.11473578],
       ...,
       [0.11521391, 0.11247651, 0.10952726, ..., 0.10835407, 0.10918588,
        0.10911623],
       [0.11367264, 0.11108444, 0.10723971, ..., 0.10690491, 0.10693726,
        0.10876018],
       [0.1121679 , 0.11034484, 0.10784435, ..., 0.10717932, 0.10929499,
        0.11235355]])

In [681]:
import numpy as np

# This is a placeholder for the actual sub-Gaussian norm computation.
def subgaussian_norm(X):
    # You would replace this with the actual calculation, which depends on the distribution of X.
    # For example, for a standard normal variable, the sub-Gaussian norm is 1.
    return 1

def calculate_K(A):
    # Assuming A is a 2D numpy array of random variables.
    subgaussian_norms = np.array([[subgaussian_norm(A[i, j]) for j in range(A.shape[1])] for i in range(A.shape[0])])
    K = np.max(subgaussian_norms)
    return K

# Example usage:
# A = np.random.randn(m, n)  # This would be your m x n random matrix with sub-Gaussian entries.
# K = calculate_K(A)
# print(K)

In [682]:
calculate_K(img[0])

1

In [683]:
d = 64
gamma = 0.5
m = img.shape[0]
r = np.max(np.linalg.norm(img, axis=(1,2)))
print(r)
c = r * (np.max([np.sqrt((d + np.log(4/gamma))/m), (d + np.log(4/gamma))/m]) + np.sqrt((d + np.log(4/gamma))/m))
c

57.54677765281298


29.585859319877713

In [684]:
# c = 1
v = 0.5
beta = 0.5
ΣB = confidence_noise_determination(img, c, v, beta, r)

In [685]:
ΣB

array([[1899.85194276,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        , 1899.85194276,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        , 1899.85194276, ...,    0.        ,
           0.        ,    0.        ],
       ...,
       [   0.        ,    0.        ,    0.        , ..., 1899.85194276,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
        1899.85194276,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        , 1899.85194276]])

In [686]:
ΣB.shape

(64, 64)

In [687]:
B = np.random.multivariate_normal(np.zeros(d), ΣB, m)

In [688]:
B

array([[ 33.04917245,  26.54839477,  15.77527837, ...,  67.87171697,
         -8.93766856, -37.20861235],
       [ 32.20830614, -56.44977472, -32.94790901, ...,   4.17772473,
        -14.11902775, -68.28695503],
       [ 39.394319  , -80.53360138,  94.60407306, ...,  34.81565998,
         16.00586369,  26.67996681],
       ...,
       [-52.42258874,  74.14701231, -59.80421492, ..., -31.78854223,
        -47.20842808,  38.91407245],
       [-36.0484389 , -56.65460679,  42.78897879, ...,  17.14727832,
        -56.79538053, -68.80298049],
       [  1.10826324,  97.12303954,  26.54767274, ...,  38.24023198,
        -69.08273574,  51.86546243]])

In [689]:
img[0] + B[0]

array([[ 33.04917245,  26.54839477,  15.80272935, ...,  67.98936403,
         -8.86315876, -37.02429862],
       [ 33.04917245,  26.54839477,  15.81449405, ...,  67.94622678,
         -8.84747248, -36.90273   ],
       [ 33.04917245,  26.54839477,  15.83802346, ...,  68.3187758 ,
         -8.65923719, -36.82037705],
       ...,
       [ 33.05701559,  26.55623791,  15.86155288, ...,  68.17759933,
         -8.72590386, -36.96939666],
       [ 33.06093716,  26.56015948,  15.83018033, ...,  68.14230521,
         -8.80041366, -37.06351431],
       [ 33.080545  ,  26.57192418,  15.80665091, ...,  68.17759933,
         -8.74159013, -36.96155352]])

In [690]:
np.mean(np.linalg.norm(B, axis=1))

347.3018524820325